# Discrete Logarithm Problem

In [ ]:
import math
import numpy as np
import secrets
import sys, os
from typing import Optional, Set, Tuple, Generator

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'python')))
from qfl_crypto import number_theory 

## MA7010 -  Week 4 Questions

Questions should be done by hand/using a calculator except those in bold for which Maple is recommended. Early questions cover some material also delivered in MA7009.

__1.	Consider the number p = 17__

a)	Show that a = 6 is a primitive root mod 17.

b)	How many other primitive roots are there mod 17?  Once you have found one primitive root a then the Order of Powers formula (Proposition 8.3 from Stanoyevitch) means ai is also a primitive root if $gcd(I,16) = 1$.  

c)	Show that 6 generates a cyclic group when you take the operation as multiplication mod 17.

In [ ]:
len(number_theory.cyclic_group(6, 17, lambda a, b: a * b))
n = 17
print(number_theory.co_primes(n))
a_s = number_theory.co_primes(n)
g = 6
set([(k, a, g ** k % n) for a in a_s  for k in range(1, n) if g ** k % n == a]), len(set([k for k in range(1, n) for a in a_s  if g ** k % n == a ])), len(a_s)

In [ ]:
p = 17
a = 6
print([(k, a ** k % p)
       for k in range(1, p)])

print([(a, number_theory.order(a, p)) for a in range(1, p)])
print(number_theory.order_of_powers(a, p))
print(number_theory.primitive_roots(p))
len(number_theory.cyclic_group(a, p, lambda a, b: a * b)), number_theory.cyclic_group(a, p, lambda a, b: a * b)
#set([(a ** i % p) for i in range(1, p) if gcd(i, p-1) == 1])

__2.	Complete the table of logarithms modulo for p = 17 with respect to the primitive root 6 and for one other primitive element that you found in b) above.__

__3.	Consider the number m = 18.__

a)	Find φ(m) using only the definition and confirm your answer using the decomposition of m into primes. 

In [ ]:
p = 18
a = 5
print([(k, a ** k % p)
       for k in range(1, p)])

print([(a, number_theory.order(a, p)) for a in range(1, p)])
print(number_theory.order_of_powers(a, p))
print(number_theory.primitive_roots(p))
len(number_theory.cyclic_group(a, p, lambda a, b: a * b)), number_theory.cyclic_group(a, p, lambda a, b: a * b)
#set([(a ** i % p) for i in range(1, p) if gcd(i, p-1) == 1])

b)    	How many primitive roots are there for m?

c)	How big is the cyclic group that you can generate using a primitive element modulo 18 where the operation is multiplication modulo 18?

d)	What happens when you try to construct a table of logarithms modulo 18 using one of the primitive roots you found in a).

__4.	Find all numbers k < 100 that have no primitive roots; can you see any patterns? Can you prove anything you observe? __

__5.	Use the Baby Step Giant Step algorithm to find the following:__

    a)	The log18 (25) mod 37
    b)	The log5 (25) mod 37
    c)  The log5 (19) mod 43
    d)	The log20 (19) mod 43
    e)	The log39 (47) mod 113
    f)	The log15 (67) mod 97

Using the fact that 18 = 57 mod 37 could you find the answer to b) using just this fact and your answer to a)? In general, therefore, how is your answer to b) related to a) and your answer to c) related to d) (if at all!)? Take other values in Maple to help you see patterns if necessary. 

__6.	Try to explain in your own words by Baby Step Giant Step is a more efficient algorithm than just searching for powers. Think about the different steps and how much work they involve.__

__7. 	How would making m bigger or smaller than ceiling($\sqrt(n)$) affect the running of the algorithm – try amending the Maple procedure if it helps.__

In [ ]:
#;number_theory.order(4, 7
n = 601 # 334 # 25 # 334 # 109 # 
ord_n = number_theory.totient(n)
ds = number_theory.divisors(ord_n)

print(ds)
#print([d - d_1 for d, d_1 in zip(ds, [0,] + ds[:-1])])
#print([(k, 7 ** k, (7 ** k) % n, 7 ** int(k), (7 ** int(k)) % n) for k in ds])
#[1, 2, 3, 4, 5, 6, 8, 10, 12, 15, 20, 24, 25, 30, 40, 50, 60, 75, 100, 120, 150, 200, 300, 600]
print(number_theory.order(2, n), number_theory.is_order_n(2, n))
print(number_theory.order(7, n), number_theory.is_order_n(7, n))
print(3, 3 ** 600 % n,  number_theory.order(3, n))
print(number_theory.is_order_n(3, n))
print(7,  7 ** 600 % n,  number_theory.order(7, n))
print(number_theory.is_order_n(7, n))
#for a in co_primes(n):
#    if number_theory.order(a, n) == ord_n:
#        print(a)
# [7, 11, 14, 19, 22, 29, 33, 34, 35, 38, 41, 42, 43, 44, 53, 55, 56, 58, 66, 68, 70, 73, 82, 84, 86, 88, 91, 93, 95, 103, 107, 110, 112, 114, 127, 136, 137, 142, 143, 145, 146, 152, 153, 155, 159, 161, 164, 168, 172, 173, 174, 177, 182, 186, 189, 190, 206, 212, 220, 224, 228, 232, 247, 249, 253, 254, 255, 257, 264, 265, 272, 274, 280, 283, 284, 286, 290, 291, 292, 297, 304, 309, 310, 311, 315, 317, 318, 321, 327, 329, 336, 337, 344, 346, 347, 348, 352, 354, 369, 373, 377, 381, 389, 395, 411, 412, 415, 419, 424, 427, 428, 429, 433, 437, 440, 442, 446, 448, 449, 455, 456, 458, 459, 464, 465, 474, 487, 489, 491, 494, 498, 506, 508, 510, 513, 515, 517, 519, 528, 531, 533, 535, 543, 545, 546, 548, 557, 558, 559, 560, 563, 566, 567, 568, 572, 579, 582, 587, 590, 594]
#
print(number_theory.primitive_roots(n))
print(number_theory.order_of_powers(7, n))

In [ ]:
n = 601
ord_n = number_theory.totient(n)
fs = [f[0] for f in number_theory.ifactors(ord_n)] + [ord_n, ]

print(sorted(set(number_theory.order(a, n) for a in range(1, n)))) #, 2 ** ord_n % n
print([f for f in number_theory.ifactors(ord_n)])

a = 10
print([(a, number_theory.order(2, n)) + tuple(pow(a, k, n) for k in fs) ]) 
print((a, ) + tuple(k for k in range(1, ord_n + 1) if a ** k % n == 1)) 

print(sorted(number_theory.co_primes(number_theory.totient(n))))
print(number_theory.primitive_roots(n))

In [ ]:
print(number_theory.primitive_roots(25))


In [ ]:
f_p = np.array([1, 2, 3, 0])
(f_p + 1)[0-3:], (f_p + 1)[1-3:], (f_p + 1)[2-3:]